In [14]:
import requests
import zipfile
import boto3
import os
import time
import io
import csv
from tqdm import tqdm

# --- Configuration ---
S3_BUCKET_NAME = 'epa-backups-eia'
S3_PREFIX = 'epa_ghg_tables/' 
TEMP_DIR = 'epa_temp'

# API Constants
API_BASE_URL = "https://data.epa.gov/efservice/"
METADATA_URL = "https://data.epa.gov/dmapservice/metadata.db_tables/"
CHUNK_SIZE = 9000 
RETRY_DELAY = 10 

s3_client = boto3.client('s3')

def get_ghg_table_list():
    """Fetches metadata and filters for the 'ghg' schema."""
    response = requests.get(METADATA_URL)
    response.raise_for_status()
    data = response.json()
    return [f"ghg.{item.get('TABLE_NAME', item.get('table_name')).upper()}" 
            for item in data if str(item.get('TABLE_SCHEMA', item.get('table_schema'))).lower() == 'ghg']

def process_table(qualified_table_name):
    """Paging logic that parses the CSV to count actual data rows."""
    file_id = qualified_table_name.replace('.', '_')
    local_csv = os.path.join(TEMP_DIR, f"{file_id}.csv")
    local_zip = os.path.join(TEMP_DIR, f"{file_id}.zip")
    
    has_more_data = True
    first_record = 1
    total_rows_downloaded = 0

    try:
        with open(local_csv, 'w', encoding='utf-8', newline='') as f:
            writer = None # Initialized after first successful row parse
            
            while has_more_data:
                last_record = first_record + CHUNK_SIZE - 1
                page_url = f"{API_BASE_URL}{qualified_table_name}/{first_record}:{last_record}/csv"
                
                tqdm.write(f"  REQUESTING: {page_url}")
                resp = requests.get(page_url, timeout=120)
                
                if resp.status_code == 200:
                    # Use CSV reader to parse the response
                    f_in = io.StringIO(resp.text.strip())
                    reader = list(csv.reader(f_in))
                    
                    # Row Count Analysis
                    # If first_record is 1, the first row is always the header
                    if first_record == 1:
                        data_rows = reader[1:]
                        header = reader[0]
                    else:
                        data_rows = reader
                    
                    row_count = len(data_rows)
                    tqdm.write(f"    -> Found {row_count} data rows in this chunk.")

                    if row_count > 0:
                        # Write to local file
                        csv_writer = csv.writer(f)
                        if first_record == 1:
                            csv_writer.writerow(header) # Write header only once
                        
                        csv_writer.writerows(data_rows)
                        total_rows_downloaded += row_count
                        first_record += CHUNK_SIZE
                    else:
                        # No data rows found; the table is finished
                        tqdm.write(f"    [TERMINATE] No data rows returned. Ending {qualified_table_name}.")
                        has_more_data = False
                else:
                    tqdm.write(f"    [!] HTTP {resp.status_code} - skipping table.")
                    has_more_data = False

        # Only zip and upload if rows were actually found
        if total_rows_downloaded > 0:
            with zipfile.ZipFile(local_zip, 'w', zipfile.ZIP_DEFLATED) as zf:
                zf.write(local_csv, arcname=f"{qualified_table_name}.csv")

            s3_key = f"{S3_PREFIX}{qualified_table_name}.zip"
            s3_client.upload_file(local_zip, S3_BUCKET_NAME, s3_key)
            tqdm.write(f"  [SUCCESS] {qualified_table_name} complete: {total_rows_downloaded} rows.")
        else:
            tqdm.write(f"  [SKIP] {qualified_table_name} contained no data.")

    except Exception as e:
        tqdm.write(f"  [ERROR] {qualified_table_name}: {e}")
    finally:
        if os.path.exists(local_csv): os.remove(local_csv)
        if os.path.exists(local_zip): os.remove(local_zip)

def main():
    if not os.path.exists(TEMP_DIR): os.makedirs(TEMP_DIR)
    tables = get_ghg_table_list()
    
    with tqdm(total=len(tables), desc="Overall Progress", unit="table") as pbar:
        for table in tables:
            process_table(table)
            pbar.update(1)

if __name__ == "__main__":
    main()

ERROR! Session/line number was not unique in database. History logging moved to new session 3453


Overall Progress:   0%|                              | 0/500 [00:00<?, ?table/s]

  REQUESTING: https://data.epa.gov/efservice/ghg.AA_BIO_CO2_EMISS_FACTOR/1:9000/csv


Overall Progress:   0%|                              | 0/500 [00:03<?, ?table/s]

    -> Found 2094 data rows in this chunk.
  REQUESTING: https://data.epa.gov/efservice/ghg.AA_BIO_CO2_EMISS_FACTOR/9001:18000/csv


...

    -> Found 0 data rows in this chunk.
    [TERMINATE] No data rows returned. Ending ghg.Z_SUBPART_LEVEL_INFORMATION.


Overall Progress: 100%|█████████████████▉| 499/500 [4:37:51<00:05,  5.07s/table]

  [SUCCESS] ghg.Z_SUBPART_LEVEL_INFORMATION complete: 516 rows.
  REQUESTING: https://data.epa.gov/efservice/ghg.Z_TIER4CEMS_QTRDTLS/1:9000/csv


Overall Progress: 100%|██████████████████| 500/500 [4:37:53<00:00, 33.35s/table]

  [ERROR] ghg.Z_TIER4CEMS_QTRDTLS: list index out of range


In [4]:
import boto3

S3_BUCKET_NAME = 'epa-backups-eia'
s3 = boto3.client('s3')

try:
    s3.head_bucket(Bucket=S3_BUCKET_NAME)
    print("Success: Bucket exists and is accessible!")
    
    # Try a test upload
    s3.put_object(Bucket=S3_BUCKET_NAME, Key='connection_test.txt', Body='test')
    print("Success: Test file uploaded!")
except Exception as e:
    print(f"Error: {e}")

Success: Bucket exists and is accessible!
Success: Test file uploaded!


In [15]:
import pandas as pd
import boto3
import os
from pathlib import Path

# Configuration
SOURCE_DIR = os.path.expanduser('~/Documents/epa_ghg_archive/epa_ghg_tables_all/')
S3_BUCKET = 'epa-backups-eia'
S3_PREFIX = 'epa_ghg_tables_parquet/'

# Initialize S3 Client
s3_client = boto3.client('s3')

def migrate_csv_to_s3_parquet():
    # Ensure source directory exists
    source_path = Path(SOURCE_DIR)
    if not source_path.exists():
        print(f"Error: Directory {SOURCE_DIR} not found.")
        return

    # Filter for CSV files
    csv_files = list(source_path.glob('*.csv'))
    print(f"Found {len(csv_files)} CSV files. Starting conversion and upload...")

    for file in csv_files:
        try:
            # 1. Read CSV
            df = pd.read_csv(file)
            
            # 2. Define Local and S3 paths
            parquet_filename = file.stem + '.parquet'
            local_parquet_path = source_path / parquet_filename
            s3_key = f"{S3_PREFIX}{parquet_filename}"
            
            # 3. Convert to Parquet locally
            df.to_parquet(local_parquet_path, engine='pyarrow', compression='snappy', index=False)
            
            # 4. Upload to S3
            s3_client.upload_file(str(local_parquet_path), S3_BUCKET, s3_key)
            
            # 5. Clean up local parquet file (optional)
            os.remove(local_parquet_path)
            
            print(f"Successfully processed: {file.name} -> s3://{S3_BUCKET}/{s3_key}")
            
        except Exception as e:
            print(f"Failed to process {file.name}: {str(e)}")

# Run the process
migrate_csv_to_s3_parquet()

Found 454 CSV files. Starting conversion and upload...
Successfully processed: ghg.P_SUBPART_LEVEL_INFO.csv -> s3://epa-backups-eia/epa_ghg_tables_parquet/ghg.P_SUBPART_LEVEL_INFO.parquet
Successfully processed: ghg.PUB_FACTS_AGGREGATED_EMISSION.csv -> s3://epa-backups-eia/epa_ghg_tables_parquet/ghg.PUB_FACTS_AGGREGATED_EMISSION.parquet
Successfully processed: ghg.X_MASS_BALANCE_UNIT_DETAILS.csv -> s3://epa-backups-eia/epa_ghg_tables_parquet/ghg.X_MASS_BALANCE_UNIT_DETAILS.parquet
Successfully processed: ghg.PUB_SECTOR_GHG_EMISSION_PET.csv -> s3://epa-backups-eia/epa_ghg_tables_parquet/ghg.PUB_SECTOR_GHG_EMISSION_PET.parquet
Successfully processed: ghg.W_CENTRIFUGAL_CMPS_ONSHORE.csv -> s3://epa-backups-eia/epa_ghg_tables_parquet/ghg.W_CENTRIFUGAL_CMPS_ONSHORE.parquet
Successfully processed: ghg.W_FUELS_FOR_LARGE_EXT_COMB.csv -> s3://epa-backups-eia/epa_ghg_tables_parquet/ghg.W_FUELS_FOR_LARGE_EXT_COMB.parquet
Successfully processed: ghg.Y_LOADINGOPERATIONS_INFO.csv -> s3://epa-backups-

/var/folders/71/t0hjd8994pj3nxrhb2r_15t00000gn/T/ipykernel_37427/1649524452.py:28: DtypeWarning: Columns (1,11,12,13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


Successfully processed: ghg.EF_W_LIQUIDS_UNLOAD_MISSING.csv -> s3://epa-backups-eia/epa_ghg_tables_parquet/ghg.EF_W_LIQUIDS_UNLOAD_MISSING.parquet
Successfully processed: ghg.W_UNIQUE_INJ_PUMP_BLOWDOWN.csv -> s3://epa-backups-eia/epa_ghg_tables_parquet/ghg.W_UNIQUE_INJ_PUMP_BLOWDOWN.parquet
Successfully processed: ghg.F_SMELTER_DETAILS.csv -> s3://epa-backups-eia/epa_ghg_tables_parquet/ghg.F_SMELTER_DETAILS.parquet
Successfully processed: ghg.RLPS_GHG_PROGRAM_FACILITY.csv -> s3://epa-backups-eia/epa_ghg_tables_parquet/ghg.RLPS_GHG_PROGRAM_FACILITY.parquet
Successfully processed: ghg.Z_PHOSP_AC_PROC_LINE.csv -> s3://epa-backups-eia/epa_ghg_tables_parquet/ghg.Z_PHOSP_AC_PROC_LINE.parquet
Successfully processed: ghg.W_ATMOS_TANKS_CALC_5.csv -> s3://epa-backups-eia/epa_ghg_tables_parquet/ghg.W_ATMOS_TANKS_CALC_5.parquet
Successfully processed: ghg.N_INFO.csv -> s3://epa-backups-eia/epa_ghg_tables_parquet/ghg.N_INFO.parquet
Successfully processed: ghg.R_SUBPART_LEVEL_INFORMATION.csv -> s3:/

/var/folders/71/t0hjd8994pj3nxrhb2r_15t00000gn/T/ipykernel_37427/1649524452.py:28: DtypeWarning: Columns (0,1,2,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


Failed to process ghg.PUB_FACTS_SUBP_GHG_EMISSION.csv: ("Expected bytes, got a 'float' object", 'Conversion failed for column co2e_emission with type object')
Successfully processed: ghg.Z_SUBPART_LEVEL_INFORMATION.csv -> s3://epa-backups-eia/epa_ghg_tables_parquet/ghg.Z_SUBPART_LEVEL_INFORMATION.parquet
Successfully processed: ghg.DD_SF6_PFC_EQUIPMENT_CAT.csv -> s3://epa-backups-eia/epa_ghg_tables_parquet/ghg.DD_SF6_PFC_EQUIPMENT_CAT.parquet
Successfully processed: ghg.W_WELLVENTING.csv -> s3://epa-backups-eia/epa_ghg_tables_parquet/ghg.W_WELLVENTING.parquet
Successfully processed: ghg.PUB_DIM_SUBSECTOR.csv -> s3://epa-backups-eia/epa_ghg_tables_parquet/ghg.PUB_DIM_SUBSECTOR.parquet
Successfully processed: ghg.H_NO_CEMS_RAW_MATERIAL_DETAILS.csv -> s3://epa-backups-eia/epa_ghg_tables_parquet/ghg.H_NO_CEMS_RAW_MATERIAL_DETAILS.parquet
Successfully processed: ghg.S_LIME_PRODUCT_DETAILS.csv -> s3://epa-backups-eia/epa_ghg_tables_parquet/ghg.S_LIME_PRODUCT_DETAILS.parquet
Successfully proc

...

Successfully processed: ghg.C_TIER1EQUATIONINPUT.csv -> s3://epa-backups-eia/epa_ghg_tables_parquet/ghg.C_TIER1EQUATIONINPUT.parquet
Successfully processed: ghg.W_GLYCOL_DEHY_LESS_THN_SPE_VAL.csv -> s3://epa-backups-eia/epa_ghg_tables_parquet/ghg.W_GLYCOL_DEHY_LESS_THN_SPE_VAL.parquet
Successfully processed: ghg.W_LIQUIDS_UNLOADING_CALC_ONE.csv -> s3://epa-backups-eia/epa_ghg_tables_parquet/ghg.W_LIQUIDS_UNLOADING_CALC_ONE.parquet
Successfully processed: ghg.I_FACILITY_DETAILS.csv -> s3://epa-backups-eia/epa_ghg_tables_parquet/ghg.I_FACILITY_DETAILS.parquet
Successfully processed: ghg.NN_NGL_FRACTIONATOR_METHODS.csv -> s3://epa-backups-eia/epa_ghg_tables_parquet/ghg.NN_NGL_FRACTIONATOR_METHODS.parquet
Successfully processed: ghg.EF_W_PROD_STG_TANKS_DEF.csv -> s3://epa-backups-eia/epa_ghg_tables_parquet/ghg.EF_W_PROD_STG_TANKS_DEF.parquet
Successfully processed: ghg.MV_EF_I_ANN_FGHG_PVMEMSLCD.csv -> s3://epa-backups-eia/epa_ghg_tables_parquet/ghg.MV_EF_I_ANN_FGHG_PVMEMSLCD.parquet
Succe

/var/folders/71/t0hjd8994pj3nxrhb2r_15t00000gn/T/ipykernel_37427/1649524452.py:28: DtypeWarning: Columns (0,1,2,3,4,5,6,8,9,10,12,13,14,15,17,18,19,20,21,22,23,24,29,30,31,32,34,35,36,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


Failed to process ghg.EF_W_PROD_STG_TANKS_CALC1OR2.csv: ("Expected bytes, got a 'float' object", 'Conversion failed for column annual_ch4_recovered with type object')
Successfully processed: ghg.EF_W_BLOWDOWN_STACKS_DETAILS.csv -> s3://epa-backups-eia/epa_ghg_tables_parquet/ghg.EF_W_BLOWDOWN_STACKS_DETAILS.parquet
Successfully processed: ghg.HH_WASTE_QTY_DETAILS.csv -> s3://epa-backups-eia/epa_ghg_tables_parquet/ghg.HH_WASTE_QTY_DETAILS.parquet
Successfully processed: ghg.HH_GASCOLL_MEA_LOC_DEST.csv -> s3://epa-backups-eia/epa_ghg_tables_parquet/ghg.HH_GASCOLL_MEA_LOC_DEST.parquet
Successfully processed: ghg.EF_W_PROD_STG_TANKS_MISSING.csv -> s3://epa-backups-eia/epa_ghg_tables_parquet/ghg.EF_W_PROD_STG_TANKS_MISSING.parquet
Successfully processed: ghg.RLPS_GHG_EMITTER_GAS.csv -> s3://epa-backups-eia/epa_ghg_tables_parquet/ghg.RLPS_GHG_EMITTER_GAS.parquet
Successfully processed: ghg.AA_FOSSIL_FUEL_INFORMATION.csv -> s3://epa-backups-eia/epa_ghg_tables_parquet/ghg.AA_FOSSIL_FUEL_INFORMA

/var/folders/71/t0hjd8994pj3nxrhb2r_15t00000gn/T/ipykernel_37427/1649524452.py:28: DtypeWarning: Columns (0,1,2,7,8,12,13,14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


Failed to process ghg.EF_W_EQUIP_LEAKS_ONSHORE.csv: ("Expected bytes, got a 'float' object", 'Conversion failed for column average_time_operational with type object')
Successfully processed: ghg.EF_W_ASSOCIATED_NG_DELAY.csv -> s3://epa-backups-eia/epa_ghg_tables_parquet/ghg.EF_W_ASSOCIATED_NG_DELAY.parquet
Successfully processed: ghg.N_NON_CEMS_SOURCE_INFO.csv -> s3://epa-backups-eia/epa_ghg_tables_parquet/ghg.N_NON_CEMS_SOURCE_INFO.parquet
Successfully processed: ghg.EF_W_DEHYDRATORS_LARGE.csv -> s3://epa-backups-eia/epa_ghg_tables_parquet/ghg.EF_W_DEHYDRATORS_LARGE.parquet
Successfully processed: ghg.C_BIOGENIC_CO2_DETAILS.csv -> s3://epa-backups-eia/epa_ghg_tables_parquet/ghg.C_BIOGENIC_CO2_DETAILS.parquet
Successfully processed: ghg.PP_SUBPART_LEVEL_INFORMATION.csv -> s3://epa-backups-eia/epa_ghg_tables_parquet/ghg.PP_SUBPART_LEVEL_INFORMATION.parquet
Successfully processed: ghg.Y_EQUIPMENTLEAK_INFO.csv -> s3://epa-backups-eia/epa_ghg_tables_parquet/ghg.Y_EQUIPMENTLEAK_INFO.parquet